In [1]:
# Author: Yuchen Zhou
# LMP1201 Group Project
# 2023/03/21 version 1.0
# The objective of the notebook is to load the Griffin results and store the data into a dataframe
# The dataframe will be used for machine learning

import pandas as pd
import numpy as np
import math
import glob





In [2]:
# function: getCancerType
# the function returns the cancer type if the sample is a cancer sample
# or returns healthy if the sample is a healthy sample
# param: 
#    sample_name(str) : the name of the sample
# return: 
#    cancer type or healthy(str)
def getCancerType(sample_name):
    
    start = 0
    end = 0
    
    for i in range (len(sample_name)):
        
        if start==0 and sample_name[i]=="_":
            
            start = i+1
        
        elif start!=0 and end==0 and sample_name[i]=="_":
            
            end = i
    
    return sample_name[start:end]


# function: getLabel
# the function returns the label of the sample
# if the sample is a cancer sample, the function returns 1
# if the sample is a healthy sample, the function returns 0
# param:
#    cancer_type(str): the cancer type or healthy
# return:
#    0 or 1(int)
def getLabel(cancer_type):
    
    label=None
    
    if cancer_type == "healthy":
        
        label=0
    
    else:
        
        label=1
    
    return label


# function: createFeatureDict
# the function creates a dictionary that maps feature to its information
# key is the chromosome number and the position. value is the cancer type and site's annotation
# param:
#    accessible_feature_df(data frame): a dataframe that contains all information of sites that are accessible in cancer patients
#    inaccessible_feature_df(data frame): a dataframe that contains all information of sites that are inaccessible in cancer patients
# return:
#    mappingDict(dict): a dictionary that maps key to value
def createFeatureDict(accessible_feature_df, inaccessible_feature_df):
    
    mappingDict={}
    
    for index, row in accessible_feature_df.iterrows():
        key=row["seqnames"]+'_'+str(round(row["Position"]))
        value = row["CancerType"]+'_'+row["annotation"]
        mappingDict[key]=value
        
    
    for index, row in inaccessible_feature_df.iterrows():
        key=row["seqnames"]+'_'+str(round(row["Position"]))
        value = row["Cancer Type"]+'_'+row["annotation"]
        mappingDict[key]=value
        
    return mappingDict
    

In [3]:
# function: load_data
# the function loads a set of data into a dataframe. rows are the samples and columns are the features.
# param:
#    file_names(list): the names of the files that contain data to be loaded
#    dataset_name(str): the name of the final dataset (training, validation, test)
#    feature_dict(dict): a dictionary that maps features to their information
# return:
#    df_list: a list that contains all the dataframes of individual sample 
def load_data(file_names,dataset_name,feature_dict):


    df_list=[] # create a list that contains each sample's dataframe
    row_name = [] # a list thhat contains name of the rows which are the information about features


    for file in file_names: # get each file name of the training data
    
        print(file,"is being processed") # make sure files are being processed
    
        current_file=pd.read_csv(file,sep="\t")# load each tsv file to a dataframe
        current_file=current_file.fillna(0) # fill missing values with zero
    
        # get the average of mean coverage, central coverage, and amplitude for accessible regions and inaccesible regions
    
        mean_accessible_mc=pd.DataFrame.mean(current_file[current_file.site_name=="AccessibleCan"].mean_coverage) # average of mean coverage of accesible regions
        mean_accessible_cc=pd.DataFrame.mean(current_file[current_file.site_name=="AccessibleCan"].central_coverage)# average of central coverage of accesible regions
        mean_accessible_a=pd.DataFrame.mean(current_file[current_file.site_name=="AccessibleCan"].amplitude)# average of amplitude of accesible regions

        mean_inaccessible_mc=pd.DataFrame.mean(current_file[current_file.site_name=="InaccessibleCan"].mean_coverage) # average of mean coverage of inaccesible regions
        mean_inaccessible_cc=pd.DataFrame.mean(current_file[current_file.site_name=="InaccessibleCan"].central_coverage) # average of central coverage of inaccessible regions
        mean_inaccessible_a=pd.DataFrame.mean(current_file[current_file.site_name=="InaccessibleCan"].amplitude) # average of amplitude of inaccesible regions
    
        # store sample name and cancer type
        sample_name = (current_file["sample"])[0]
        cancer_type= getCancerType(sample_name)
        
        if cancer_type =="heatlhy": # fix typos
            cancer_type = "healthy"
            
    
        # use a loop to change the name of rows (index) to be the information of each feature
        if file == file_names[0]:
            for index, row in current_file.iterrows():
        
                chrom_position = row["Chrom"]+'_'+str(row["position"])
                new_index = chrom_position+"_"+feature_dict[chrom_position]+"_"+row["site_name"]
                row_name.append(new_index)
        
        current_file.index=row_name
        
        # select mean coverage, central coverage, and amplitude
        current_df = current_file[["mean_coverage","central_coverage","amplitude"]]

        # flatten the data frame so each sample is one column in the final dataframe
        s = current_df.stack()
        flattened_current_df = pd.DataFrame([s.values],  columns=[f'{j}-{i}' for i, j in s.index])

        # create new columns to store extra information
        flattened_current_df['mean_accessible_mean_coverage']=mean_accessible_mc
        flattened_current_df['mean_accessible_central_coverage']=mean_accessible_cc
        flattened_current_df["mean_accessible_amplitude"]=mean_accessible_a

        flattened_current_df['mean_inaccessible_mean_coverage']=mean_inaccessible_mc
        flattened_current_df['mean_inaccessible_central_coverage']=mean_inaccessible_cc
        flattened_current_df["mean_inaccessible_amplitude"]=mean_inaccessible_a
    
        flattened_current_df["sample"] = sample_name # name of the sample
        flattened_current_df["cancer_type"] = cancer_type # cancer type or healthy
        flattened_current_df["label"] = getLabel(cancer_type) # 0 is healthy. 1 is cancer.
    
        df_list.append(flattened_current_df)
    
        print(len(df_list),"samples are being loaded to the final dataframe.")

    print("A total of", len(df_list), "samples are being loaded to",dataset_name )
    return df_list




In [4]:
training_files = glob.glob("Griffin Results/results_downsample*/*_train_*/*.GC_corrected*") # get the file names of all GC corrected Griffin Results for training
print("The number of files in training set is",len(training_files))

training_files_set=set(training_files)
print("The number of files with unique name in training set is",len(training_files_set))

# read the files that contain feature information about accesible sites in cancer and inaccesible site in cancer
accessibleInCancer = pd.read_csv("AccessibleInCancer.csv")
inaccessibleInCancer = pd.read_csv("InaccessibleInCancer.csv")

# create feature dictionary
FeatureDict = createFeatureDict(accessibleInCancer,inaccessibleInCancer)

df_list=load_data(training_files,"training_data", FeatureDict)

# combine all dataframes into a single dataframe
final_df = pd.concat(df_list)
final_df.index = final_df["sample"]

# store it as training data
final_df.to_csv('training_data.csv',index=False)

The number of files in training set is 719
The number of files with unique name in training set is 719
Griffin Results/results_downsample11_batch/BH01_healthy_train_s050/BH01_healthy_train_s050.GC_corrected.coverage.tsv is being processed
1 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s057/BH01_healthy_train_s057.GC_corrected.coverage.tsv is being processed
2 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s068/BH01_healthy_train_s068.GC_corrected.coverage.tsv is being processed
3 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s095/BH01_healthy_train_s095.GC_corrected.coverage.tsv is being processed
4 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s061/BH01_healthy_train_s061.GC_corrected.coverage.tsv is being processed
5 samples are

44 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s162/BH01_healthy_train_s162.GC_corrected.coverage.tsv is being processed
45 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s165/BH01_healthy_train_s165.GC_corrected.coverage.tsv is being processed
46 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s153/BH01_healthy_train_s153.GC_corrected.coverage.tsv is being processed
47 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s154/BH01_healthy_train_s154.GC_corrected.coverage.tsv is being processed
48 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s137/BH01_healthy_train_s137.GC_corrected.coverage.tsv is being processed
49 samples are being loaded to the final dataframe.
Griffin 

88 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s007/BH01_healthy_train_s007.GC_corrected.coverage.tsv is being processed
89 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s054/BH01_healthy_train_s054.GC_corrected.coverage.tsv is being processed
90 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s053/BH01_healthy_train_s053.GC_corrected.coverage.tsv is being processed
91 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s098/BH01_healthy_train_s098.GC_corrected.coverage.tsv is being processed
92 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s065/BH01_healthy_train_s065.GC_corrected.coverage.tsv is being processed
93 samples are being loaded to the final dataframe.
Griffin 

132 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s103/BH01_healthy_train_s103.GC_corrected.coverage.tsv is being processed
133 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s135/BH01_healthy_train_s135.GC_corrected.coverage.tsv is being processed
134 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s132/BH01_healthy_train_s132.GC_corrected.coverage.tsv is being processed
135 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s151/BH01_healthy_train_s151.GC_corrected.coverage.tsv is being processed
136 samples are being loaded to the final dataframe.
Griffin Results/results_downsample11_batch/BH01_healthy_train_s156/BH01_healthy_train_s156.GC_corrected.coverage.tsv is being processed
137 samples are being loaded to the final dataframe.
Gr

176 samples are being loaded to the final dataframe.
Griffin Results/results_downsample10_batch/IC16_bladder_train_s001/IC16_bladder_train_s001.GC_corrected.coverage.tsv is being processed
177 samples are being loaded to the final dataframe.
Griffin Results/results_downsample10_batch/IC06_lung_train_s001/IC06_lung_train_s001.GC_corrected.coverage.tsv is being processed
178 samples are being loaded to the final dataframe.
Griffin Results/results_downsample10_batch/IC09_ov_train_s004/IC09_ov_train_s004.GC_corrected.coverage.tsv is being processed
179 samples are being loaded to the final dataframe.
Griffin Results/results_downsample10_batch/IC09_ov_train_s003/IC09_ov_train_s003.GC_corrected.coverage.tsv is being processed
180 samples are being loaded to the final dataframe.
Griffin Results/results_downsample10_batch/IC14_head_train_s004/IC14_head_train_s004.GC_corrected.coverage.tsv is being processed
181 samples are being loaded to the final dataframe.
Griffin Results/results_downsample

...

534 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC22_lung_train_s001/IC22_lung_train_s001.GC_corrected.coverage.tsv is being processed
535 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC27_utero_train_s003/IC27_utero_train_s003.GC_corrected.coverage.tsv is being processed
536 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC27_utero_train_s004/IC27_utero_train_s004.GC_corrected.coverage.tsv is being processed
537 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC34_breast_train_s001/IC34_breast_train_s001.GC_corrected.coverage.tsv is being processed
538 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC27_utero_train_s002/IC27_utero_train_s002.GC_corrected.coverage.tsv is being processed
539 samples are being loaded to the final dataframe.
Griffin Results/results_dow

579 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC15_lung_train_s054/IC15_lung_train_s054.GC_corrected.coverage.tsv is being processed
580 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s006/IC17_liver_train_s006.GC_corrected.coverage.tsv is being processed
581 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC15_lung_train_s053/IC15_lung_train_s053.GC_corrected.coverage.tsv is being processed
582 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s039/IC17_liver_train_s039.GC_corrected.coverage.tsv is being processed
583 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC15_lung_train_s071/IC15_lung_train_s071.GC_corrected.coverage.tsv is being processed
584 samples are being loaded to the final dataframe.
Griffin Results/results_downsampl

624 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s047/IC17_liver_train_s047.GC_corrected.coverage.tsv is being processed
625 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC15_lung_train_s012/IC15_lung_train_s012.GC_corrected.coverage.tsv is being processed
626 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s078/IC17_liver_train_s078.GC_corrected.coverage.tsv is being processed
627 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s003/IC17_liver_train_s003.GC_corrected.coverage.tsv is being processed
628 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC15_lung_train_s069/IC15_lung_train_s069.GC_corrected.coverage.tsv is being processed
629 samples are being loaded to the final dataframe.
Griffin Results/results_downsam

669 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s060/IC17_liver_train_s060.GC_corrected.coverage.tsv is being processed
670 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s094/IC17_liver_train_s094.GC_corrected.coverage.tsv is being processed
671 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s042/IC17_liver_train_s042.GC_corrected.coverage.tsv is being processed
672 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC15_lung_train_s028/IC15_lung_train_s028.GC_corrected.coverage.tsv is being processed
673 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC15_lung_train_s017/IC15_lung_train_s017.GC_corrected.coverage.tsv is being processed
674 samples are being loaded to the final dataframe.
Griffin Results/results_downsam

714 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s019/IC17_liver_train_s019.GC_corrected.coverage.tsv is being processed
715 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s026/IC17_liver_train_s026.GC_corrected.coverage.tsv is being processed
716 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC15_lung_train_s080/IC15_lung_train_s080.GC_corrected.coverage.tsv is being processed
717 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC15_lung_train_s074/IC15_lung_train_s074.GC_corrected.coverage.tsv is being processed
718 samples are being loaded to the final dataframe.
Griffin Results/results_downsample2_batch/IC17_liver_train_s021/IC17_liver_train_s021.GC_corrected.coverage.tsv is being processed
719 samples are being loaded to the final dataframe.
A total of 719 samples are bein

In [5]:
final_df

mean_coverage-chr1_1432046_BRCA_Promoter_AccessibleCan  \
sample                                                                            
BH01_healthy_train_s050                                            1.30591        
BH01_healthy_train_s057                                            0.48061        
BH01_healthy_train_s068                                            1.39603        
BH01_healthy_train_s095                                            0.38662        
BH01_healthy_train_s061                                            2.04638        
...                                                                    ...        
IC17_liver_train_s019                                              0.71153        
IC17_liver_train_s026                                              1.20549        
IC15_lung_train_s080                                               0.88148        
IC15_lung_train_s074                                               0.71708        
IC17_liver_train_s021                                              0.74276        

                         central_coverage-chr1_1432046_BRCA_Promoter_AccessibleCan  \
sample                                                                               
BH01_healthy_train_s050                                            0.00000           
BH01_healthy_train_s057                                            0.00000           
BH01_healthy_train_s068                                           -0.94532           
BH01_healthy_train_s095                                            0.00000           
BH01_healthy_train_s061                                            8.57232           
...                                                                    ...           
IC17_liver_train_s019                                              0.00000           
IC17_liver_train_s026                                             -0.60874           
IC15_lung_train_s080                                               0.00000           
IC15_lung_train_s074                                               0.00000           
IC17_liver_train_s021                                              0.00000           

                         amplitude-chr1_1432046_BRCA_Promoter_AccessibleCan  \
sample                                                                        
BH01_healthy_train_s050                                           39.07838    
BH01_healthy_train_s057                                           50.77401    
BH01_healthy_train_s068                                           57.11249    
BH01_healthy_train_s095                                           43.94859    
BH01_healthy_train_s061                                           73.02630    
...                                                                    ...    
IC17_liver_train_s019                                             51.91221    
IC17_liver_train_s026                                             35.32616    
IC15_lung_train_s080                                              47.85163    
IC15_lung_train_s074                                              53.03949    
IC17_liver_train_s021                                             23.59569    

                         mean_coverage-chr1_1759034_BRCA_Exon_AccessibleCan  \
sample                                                                        
BH01_healthy_train_s050                                            0.69223    
BH01_healthy_train_s057                                            1.38140    
BH01_healthy_train_s068                                            0.00000    
BH01_healthy_train_s095                                            1.69048    
BH01_healthy_train_s061                                            1.79420    
...                                                                    ...    
IC17_liver_train_s019                                              0.54558    
IC17_liver_train_s026                                              0.80275    
IC15_lun

In [6]:
validation_files = glob.glob("Griffin Results/results_downsample*/*_val_*/*.GC_corrected*") # get the file names of all GC corrected Griffin Results for training
print("The number of files in validation set is",len(validation_files))

validation_files_set=set(validation_files)
print("The number of files with unique name in validation set is",len(validation_files_set))

df_list=load_data(validation_files,"validation_data", FeatureDict)

# combine all dataframes into a single dataframe
final_df = pd.concat(df_list)
final_df.index = final_df["sample"]

# store it as training data
final_df.to_csv('validation_data.csv',index=False)

The number of files in validation set is 101
The number of files with unique name in validation set is 101
Griffin Results/results_downsample8_batch/IH02_healthy_val_s063/IH02_healthy_val_s063.GC_corrected.coverage.tsv is being processed
1 samples are being loaded to the final dataframe.
Griffin Results/results_downsample8_batch/IH02_healthy_val_s052/IH02_healthy_val_s052.GC_corrected.coverage.tsv is being processed
2 samples are being loaded to the final dataframe.
Griffin Results/results_downsample8_batch/IH02_healthy_val_s055/IH02_healthy_val_s055.GC_corrected.coverage.tsv is being processed
3 samples are being loaded to the final dataframe.
Griffin Results/results_downsample8_batch/IH02_healthy_val_s001/IH02_healthy_val_s001.GC_corrected.coverage.tsv is being processed
4 samples are being loaded to the final dataframe.
Griffin Results/results_downsample8_batch/IH02_healthy_val_s039/IH02_healthy_val_s039.GC_corrected.coverage.tsv is being processed
5 samples are being loaded to the 

45 samples are being loaded to the final dataframe.
Griffin Results/results_downsample8_batch/IH02_healthy_val_s057/IH02_healthy_val_s057.GC_corrected.coverage.tsv is being processed
46 samples are being loaded to the final dataframe.
Griffin Results/results_downsample8_batch/IH02_healthy_val_s061/IH02_healthy_val_s061.GC_corrected.coverage.tsv is being processed
47 samples are being loaded to the final dataframe.
Griffin Results/results_downsample8_batch/IH02_healthy_val_s059/IH02_healthy_val_s059.GC_corrected.coverage.tsv is being processed
48 samples are being loaded to the final dataframe.
Griffin Results/results_downsample8_batch/IH03_healthy_val_s002/IH03_healthy_val_s002.GC_corrected.coverage.tsv is being processed
49 samples are being loaded to the final dataframe.
Griffin Results/results_downsample8_batch/IH02_healthy_val_s044/IH02_healthy_val_s044.GC_corrected.coverage.tsv is being processed
50 samples are being loaded to the final dataframe.
Griffin Results/results_downsampl

90 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC32_lung_val_s004/IC32_lung_val_s004.GC_corrected.coverage.tsv is being processed
91 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC28_lung_val_s002/IC28_lung_val_s002.GC_corrected.coverage.tsv is being processed
92 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC28_lung_val_s003/IC28_lung_val_s003.GC_corrected.coverage.tsv is being processed
93 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC28_lung_val_s004/IC28_lung_val_s004.GC_corrected.coverage.tsv is being processed
94 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC24_lpan_val_s001/IC24_lpan_val_s001.GC_corrected.coverage.tsv is being processed
95 samples are being loaded to the final dataframe.
Griffin Results/results_downsample4_batch/IC32_lung_val_s001/IC

In [7]:
final_df

mean_coverage-chr1_1432046_BRCA_Promoter_AccessibleCan  \
sample                                                                          
IH02_healthy_val_s063                                            0.78331        
IH02_healthy_val_s052                                            0.72621        
IH02_healthy_val_s055                                            0.59689        
IH02_healthy_val_s001                                            0.86836        
IH02_healthy_val_s039                                            0.53150        
...                                                                  ...        
IC23_liver_val_s001                                              0.81072        
IC28_lung_val_s001                                               1.34710        
IC24_lpan_val_s002                                               1.27692        
IC24_lpan_val_s004                                               0.93068        
IC24_lpan_val_s003                                               1.35312        

                       central_coverage-chr1_1432046_BRCA_Promoter_AccessibleCan  \
sample                                                                             
IH02_healthy_val_s063                                            0.24132           
IH02_healthy_val_s052                                           -0.55852           
IH02_healthy_val_s055                                           -0.54958           
IH02_healthy_val_s001                                            3.06452           
IH02_healthy_val_s039                                            0.00000           
...                                                                  ...           
IC23_liver_val_s001                                              0.00000           
IC28_lung_val_s001                                               0.90716           
IC24_lpan_val_s002                                               1.34938           
IC24_lpan_val_s004                                               1.30085           
IC24_lpan_val_s003                                               1.18028           

                       amplitude-chr1_1432046_BRCA_Promoter_AccessibleCan  \
sample                                                                      
IH02_healthy_val_s063                                           21.88472    
IH02_healthy_val_s052                                           36.67839    
IH02_healthy_val_s055                                           63.39674    
IH02_healthy_val_s001                                           87.90771    
IH02_healthy_val_s039                                           26.90275    
...                                                                  ...    
IC23_liver_val_s001                                             27.22859    
IC28_lung_val_s001                                              95.47706    
IC24_lpan_val_s002                                              84.60637    
IC24_lpan_val_s004                                              68.68037    
IC24_lpan_val_s003                                              95.37910    

                       mean_coverage-chr1_1759034_BRCA_Exon_AccessibleCan  \
sample                                                                      
IH02_healthy_val_s063                                            0.61875    
IH02_healthy_val_s052                                            0.92918    
IH02_healthy_val_s055                                            1.19165    
IH02_healthy_val_s001                                            0.42336    
IH02_healthy_val_s039                                            0.38354    
...                                                                  ...    
IC23_liver_val_s001                                              0.43225    
IC28_lung_val_s001                                               0.84717    
IC24_lpan_val_s002                                               1.17374    
IC24_lpan_val_s004         

In [8]:
test_files = glob.glob("Griffin Results/results_downsample*/*_test_*/*.GC_corrected*") # get the file names of all GC corrected Griffin Results for training
print("The number of files in test set is",len(test_files))

test_files_set=set(test_files)
print("The number of files with unique name in test set is",len(test_files_set))

df_list=load_data(test_files,"test_data", FeatureDict)

# combine all dataframes into a single dataframe
final_df = pd.concat(df_list)
final_df.index = final_df["sample"]

# store it as training data
final_df.to_csv('test_data.csv',index=False)

The number of files in test set is 211
The number of files with unique name in test set is 211
Griffin Results/results_downsample10_batch/IC19_balls_test_s001/IC19_balls_test_s001.GC_corrected.coverage.tsv is being processed
1 samples are being loaded to the final dataframe.
Griffin Results/results_downsample10_batch/IC05_lung_test_s002/IC05_lung_test_s002.GC_corrected.coverage.tsv is being processed
2 samples are being loaded to the final dataframe.
Griffin Results/results_downsample10_batch/IC08_utero_test_s001/IC08_utero_test_s001.GC_corrected.coverage.tsv is being processed
3 samples are being loaded to the final dataframe.
Griffin Results/results_downsample10_batch/IC04_breast_test_s001/IC04_breast_test_s001.GC_corrected.coverage.tsv is being processed
4 samples are being loaded to the final dataframe.
Griffin Results/results_downsample10_batch/IC03_skin_test_s004/IC03_skin_test_s004.GC_corrected.coverage.tsv is being processed
5 samples are being loaded to the final dataframe.
Gr

45 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s030/IH01_heatlhy_test_s030.GC_corrected.coverage.tsv is being processed
46 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s006/IH01_heatlhy_test_s006.GC_corrected.coverage.tsv is being processed
47 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s039/IH01_heatlhy_test_s039.GC_corrected.coverage.tsv is being processed
48 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s001/IH01_heatlhy_test_s001.GC_corrected.coverage.tsv is being processed
49 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s096/IH01_heatlhy_test_s096.GC_corrected.coverage.tsv is being processed
50 samples are being loaded to the final dataframe.
Griffin Results/re

90 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s135/IH01_heatlhy_test_s135.GC_corrected.coverage.tsv is being processed
91 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s103/IH01_heatlhy_test_s103.GC_corrected.coverage.tsv is being processed
92 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s104/IH01_heatlhy_test_s104.GC_corrected.coverage.tsv is being processed
93 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s122/IH01_heatlhy_test_s122.GC_corrected.coverage.tsv is being processed
94 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s125/IH01_heatlhy_test_s125.GC_corrected.coverage.tsv is being processed
95 samples are being loaded to the final dataframe.
Griffin Results/re

134 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s010/IH01_heatlhy_test_s010.GC_corrected.coverage.tsv is being processed
135 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s028/IH01_heatlhy_test_s028.GC_corrected.coverage.tsv is being processed
136 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s017/IH01_heatlhy_test_s017.GC_corrected.coverage.tsv is being processed
137 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s021/IH01_heatlhy_test_s021.GC_corrected.coverage.tsv is being processed
138 samples are being loaded to the final dataframe.
Griffin Results/results_downsample12_batch/IH01_heatlhy_test_s019/IH01_heatlhy_test_s019.GC_corrected.coverage.tsv is being processed
139 samples are being loaded to the final dataframe.
Griffin Resu

179 samples are being loaded to the final dataframe.
Griffin Results/results_downsample9_batch/IC50_pan_test_s001/IC50_pan_test_s001.GC_corrected.coverage.tsv is being processed
180 samples are being loaded to the final dataframe.
Griffin Results/results_downsample9_batch/IC50_pan_test_s006/IC50_pan_test_s006.GC_corrected.coverage.tsv is being processed
181 samples are being loaded to the final dataframe.
Griffin Results/results_downsample9_batch/IC50_pan_test_s007/IC50_pan_test_s007.GC_corrected.coverage.tsv is being processed
182 samples are being loaded to the final dataframe.
Griffin Results/results_downsample9_batch/IC50_pan_test_s009/IC50_pan_test_s009.GC_corrected.coverage.tsv is being processed
183 samples are being loaded to the final dataframe.
Griffin Results/results_downsample9_batch/50_pan_test_s002/50_pan_test_s002.GC_corrected.coverage.tsv is being processed
184 samples are being loaded to the final dataframe.
Griffin Results/results_downsample9_batch/IC52_pan_test_s011/

In [9]:
final_df

mean_coverage-chr1_1432046_BRCA_Promoter_AccessibleCan  \
sample                                                                              
IC19_balls_test_s001                                                 1.80369        
IC05_lung_test_s002                                                  0.50383        
IC08_utero_test_s001                                                 0.90305        
IC04_breast_test_s001                                                0.46883        
IC03_skin_test_s004                                                  0.55954        
...                                                                      ...        
IC29_head_test_s002                                                  0.56453        
IC29_head_test_s003                                                  1.00505        
IC29_head_test_s004                                                  1.34844        
IC33_colorectal_test_s001                                            0.46115        
IC30_esophageal_test_s001                                            1.13717        

                           central_coverage-chr1_1432046_BRCA_Promoter_AccessibleCan  \
sample                                                                                 
IC19_balls_test_s001                                                 0.00000           
IC05_lung_test_s002                                                  0.00000           
IC08_utero_test_s001                                                 4.19543           
IC04_breast_test_s001                                                0.00000           
IC03_skin_test_s004                                                  0.00000           
...                                                                      ...           
IC29_head_test_s002                                                  0.00000           
IC29_head_test_s003                                                  0.00000           
IC29_head_test_s004                                                 -0.41545           
IC33_colorectal_test_s001                                            0.00000           
IC30_esophageal_test_s001                                            0.25730           

                           amplitude-chr1_1432046_BRCA_Promoter_AccessibleCan  \
sample                                                                          
IC19_balls_test_s001                                                37.72721    
IC05_lung_test_s002                                                 21.38752    
IC08_utero_test_s001                                                64.63931    
IC04_breast_test_s001                                               48.54248    
IC03_skin_test_s004                                                 44.78269    
...                                                                      ...    
IC29_head_test_s002                                                 30.61135    
IC29_head_test_s003                                                 75.85494    
IC29_head_test_s004                                                 72.02165    
IC33_colorectal_test_s001                                           35.81013    
IC30_esophageal_test_s001                                           58.17956    

                           mean_coverage-chr1_1759034_BRCA_Exon_AccessibleCan  \
sample                                                                          
IC19_balls_test_s001                                                 1.20808    
IC05_lung_test_s002                                                  0.72189    
IC08_utero_test_s001                                                 0.32840    
IC04_breast_test_s001                                                1.20601    
IC03_skin_test_s004                                                  0.78480    
...                                                                      ...    
IC29_head_test_s002                                                  1.443